In [1]:
## show dataset 

In [11]:
import pandas as pd
import numpy as np





In [5]:
data_path = "../tcdata/"
save_path = "../savepath/"

articles = pd.read_csv(data_path + "articles.csv")
testA_click_log = pd.read_csv(data_path + "testA_click_log.csv")
train_click_log = pd.read_csv(data_path + "train_click_log.csv")

In [6]:
articles.head()

,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [7]:
train_click_log.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [8]:
testA_click_log.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


In [13]:
# 小样本数据
def get_all_click_sample(data_path="../tcdata/", sample_nums = 10000):
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False)
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]

    # 时间归一化，计算权重
    max_min_scaler = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
    all_click['click_timestamp'] = all_click[['click_timestamp']].apply(max_min_scaler)

    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict


click_df  = get_all_click_sample(data_path)
click_df 

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
24,199992,272143,0.000015,4,1,17,1,25,2
25,199992,348111,0.000024,4,1,17,1,25,2
26,199992,299697,0.000039,4,1,17,1,25,2
154,199950,182513,0.000172,4,1,17,1,16,2
155,199950,272143,0.000370,4,1,17,1,16,2
...,...,...,...,...,...,...,...,...,...
1112508,132251,16346,0.587351,4,1,17,1,20,1
1112540,65307,76012,0.587290,4,3,20,1,25,1
1112541,65307,70986,0.587305,4,3,20,1,25,1
1112597,3,50644,0.587402,4,3,2,1,25,2


In [16]:
get_user_item_time(click_df)

C:\Users\zhunian1\AppData\Local\Temp\ipykernel_16660\2939100957.py:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\


{3: [(50644, 0.5874020621165293), (36162, 0.5874169745052613)],
 17: [(156279, 0.5872859801060025),
  (158331, 0.587429028686153),
  (363916, 0.587443941074885)],
 56: [(36162, 0.5868990041154872), (294107, 0.5869139165042192)],
 88: [(36162, 0.5861516463914416), (277107, 0.5861665587801737)],
 123: [(205824, 0.5858932803092268), (50644, 0.5859081926979588)],
 126: [(224730, 0.5857979354664707), (223926, 0.5858128478552027)],
 142: [(248438, 0.5856671776771132), (249524, 0.5856820900658453)],
 185: [(36162, 0.5855488662644711), (211442, 0.5855637786532031)],
 189: [(107190, 0.5852163513117642),
  (107014, 0.5852462919087806),
  (331116, 0.585378283449766),
  (184033, 0.5854751672510391),
  (129237, 0.5854900796397711)],
 200: [(285433, 0.5852388481414053), (16346, 0.5852537605301372)],
 248: [(36162, 0.5848218425766207), (211442, 0.5848367549653527)],
 276: [(50644, 0.58447273709437), (166322, 0.584487649483102)],
 286: [(270742, 0.584740822077401),
  (270175, 0.5848804691416817),
  (2